In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

USE_CUDA = False

In [3]:
from nalgene.generate import *

parsed = parse_file('.', 'grammar.nlg')
parsed.map_leaves(tokenizeLeaf)

In [4]:
def descend(node, fn, child_type='phrase', returns=None):
    if returns is None: returns = []
    returned = fn(node)
    returns.append(returned)

    for child in node.children:
        if (child_type is None) or (child.type == child_type):
            descend(child, fn, child_type, returns)
    
    return returns

In [5]:
def ascend(node, fn):
    if node.parent is None:
        return fn(node)
    else:
        return ascend(node.parent, fn)

## Building input and output vocabularies

To find all input vocabulary tokens, we can traverse the parsed nalgene tree and copy all `word` type tokens.

**TODO**: Use GloVe vectors for input vocabulary

In [6]:
input_tokens = []

def get_input_tokens(node):
    if node.type == 'word':
        input_tokens.append(node.key)

descend(parsed, get_input_tokens, None)

input_tokens = list(set(input_tokens))
input_tokens = ['EOS'] + input_tokens
print(input_tokens)

['EOS', 'high', 'Sexbomb', 'For', 'Bitch', 'Skrillex', 'Breath', 'Own', 'Way', 'Sympathy', 'Waterloo', 'hey', 'Space', 'less', 'Sunset', 'Can', 'Breeze', 'Cover', 'Album', 'Maroon', 'Summer', 'microsoft', 'and', 'Darkness', 'Let', 'Machine', 'Dolapdere', 'Love', 'eth', "what's", 'hot', 'by', 'Smack', 'Peaches', 'Sezen', 'Seaside', 'Nite', 'Ceza', 'The', 'Modern', 'um', 'seconds', 'Louie', 'Boogie', 'music', 'You', 'Jovi', 'Shut', 'Omen', 'Badem', 'white', 'Firestarter', 'plox', 'Heart', 'equal', 'pm', 'Gimme', 'Tonight', 'Voodoo', 'basement', 'Turning', 'Shot', 'Natalie', 'Morrissey', 'Together', 'becomes', 'Human', 'hello', 'Up', 'Did', 'the', 'A', 'Will', 'Full', 'Daydream', 'F', 'song', 'change', '5', 'orange', 'Rappers', 'Remaster', 'Sex', 'maia', 'Berlin', 'Someday', 'Disposition', 'Melody', 'New', 'coffee', 'White', 'Sweet', 'Placebo', 'Aerosmith', 'Ferah', 'Her', 'also', 'Believer', 'then', 'Imbruglia', 'Edit', 'Delight', 'Pants', 'there', 'to', 'if', 'of', 'Time', 'Eyes', 'very

For output tokens, we can just take the top level node names that are either phrases or variables.

In [7]:
output_tokens = [child.key for child in parsed.children if child.type in ['phrase', 'variable']]
output_tokens = ['EOS'] + output_tokens
print(output_tokens)

['EOS', '%', '%if', '%timer', '%sequence', '%time', '%relative_time', '%absolute_time', '%condition', '%checkValue', '%getValue', '%action', '%checkLightState', '%checkSwitchState', '%getLightState', '%getSwitchState', '%getTemperature', '%getPrice', '%setLightState', '%light_state', '%setSwitchState', '%setTemperature', '%setVolume', '%playMusic', '$operator', '$asset', '$room_name', '$light_name', '$switch_name', '$on_off', '$up_down', '$color', '$number', '$_number', '$time_unit', '$temperature', '$time', '$artist_name', '$song_name']


## Getting input and target data for nodes

In [8]:
def words_for_position(words, position):
    if position is None:
        return words
    start, end, length = position
    return words[start : end + 1]

In [9]:
def relative_position(node, parent):
    if parent.position is None:
        return node.position
    return node.position[0] - parent.position[0], node.position[1] - parent.position[0], node.position[2]

In [10]:
def data_for_node(flat, node):
    words = [child.key for child in flat.children]
    inputs = words_for_position(words, node.position)
    keys = [child.key for child in node.children]
    positions = [relative_position(child, node) for child in node.children]
    return node.key, inputs, list(zip(keys, positions))

In [11]:
walked_flat, walked_tree = walk_tree(parsed, parsed['%'], None)
data_for_node(walked_flat, walked_tree.children[0])

('%sequence',
 ['plx',
  'the',
  'kitchen',
  'temperature',
  'and',
  'also',
  'turn',
  'the',
  'temperature',
  'of',
  'kitchen',
  'to',
  '1',
  'C',
  'and',
  'could',
  'you',
  'also',
  'plox',
  'set',
  'the',
  'office',
  'light',
  'blue'],
 [('%action', (1, 3, 3)), ('%action', (6, 13, 8)), ('%action', (19, 23, 5))])

## Creating tensors for input and target data

In [12]:
def tokens_to_tensor(tokens, source_tokens, append_eos=True):
    indexes = []
    for token in tokens:
        indexes.append(source_tokens.index(token))
    if append_eos:
        indexes.append(0)
    return torch.LongTensor(indexes)

In [13]:
def ranges_to_tensor(ranges, seq_len):
    ranges_tensor = torch.zeros(len(ranges), seq_len)
    for r in range(len(ranges)):
        start, end, _ = ranges[r]
        ranges_tensor[r, start:end+1] = 1
    return ranges_tensor

ranges_to_tensor([(0, 2, 3), (4, 4, 1)], 5)


 1  1  1  0  0
 0  0  0  0  1
[torch.FloatTensor of size 2x5]

## Model

The core model is a regular seq2seq/encoder-decoder model with attention. The attention model is from [Luong et al.'s "Effective Approaches to Attention-based Neural Machine Translation"](https://arxiv.org/abs/1508.04025) using dot-product based attention energies, with one important difference: there is no softmax layer, allowing attention to focus on multiple tokens at once. Instead a sigmoid layer is added to squeeze outputs between 0 and 1.

The encoder and decoder take one additional input `context` which represents the type of phrase, e.g. `%setLightState`. At the top level node the context is always `%`.

The encoder encodes the input sequence into a series of vectors using a bidirectional GRU. The decoder "translates" this into a sequence of phrase tokens, given the encoder outputs and current context, e.g. "turn off the office light" + `%setLightState` &rarr; `[$on_off, $light]`.

Once the decoder has chosen tokens and alignments, the phrase tokens and selection of inputs are used as the context and inputs of the next iteration. This recurs until no more phrase tokens are found.

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, bidirectional=True)

    def forward(self, context_input, word_inputs):
        # TODO: Incorporate context input
        # TODO: Batching
        
        seq_len = word_inputs.size(0)
        batch_size = word_inputs.size(1)
        
        embedded = self.embedding(word_inputs.view(seq_len * batch_size, -1)) # Process seq x batch at once
        output = embedded.view(seq_len, batch_size, -1) # Resize back to seq x batch for RNN

        outputs, hidden = self.gru(output)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs

        return outputs, hidden

In [15]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, hidden, encoder_outputs):
        seq_len = len(encoder_outputs)

        # Create variable to store attention energies
        attention_energies = Variable(torch.zeros(seq_len)) # B x 1 x S
        if USE_CUDA: attention_energies = attention_energies.cuda()

        # Calculate energies for each encoder output
        for i in range(seq_len):
            attention_energies[i] = hidden.dot(encoder_outputs[i])

        # Squeeze to range 0 to 1, resize to 1 x 1 x seq_len
        return F.sigmoid(attention_energies).unsqueeze(0).unsqueeze(0)

In [16]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout=0.05):
        super(Decoder, self).__init__()
        
        # Keep parameters for reference
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout)
        self.out = nn.Linear(hidden_size * 2, output_size)
        
        # Attention module
        self.attention = Attention()
    
    def forward(self, context_input, word_input, last_hidden, encoder_outputs):
        # Note: we run this one step at a time
        # TODO: Batching
        
        # Get the embedding of the current input word (last output word)
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N
        
        # Combine context and embedded word, through RNN
        rnn_input = torch.cat((context_input.unsqueeze(0), word_embedded), 2)
        rnn_output, hidden = self.gru(rnn_input, last_hidden)

        # Calculate attention from current RNN state and all encoder outputs; apply to encoder outputs
        attention_weights = self.attention(rnn_output.squeeze(0), encoder_outputs)
        context = attention_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
        
        # Final output layer (next word prediction) using the RNN hidden state and context vector
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        output = F.log_softmax(self.out(torch.cat((rnn_output, context), 1)))
        
        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attention_weights

We can turn the whole thing into one module by combining the Encoder and Decoder networks. There's also an embedding layer for the context tokens.

In [17]:
MAX_LENGTH = 50

class RARNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(RARNN, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        
        self.encoder = Encoder(input_size, hidden_size)
        self.decoder = Decoder(hidden_size, output_size)

    def forward(self, context_input, word_inputs, word_targets=None):
        # Get embedding for context input
        context_embedded = self.embedding(context_input)
        
        input_len = word_inputs.size(0)
        target_len = word_targets.size(0) if word_targets is not None else MAX_LENGTH
        
        # Run through encoder
        encoder_outputs, encoder_hidden = self.encoder(context_embedded, word_inputs)
        decoder_hidden = encoder_hidden # Use encoder's last hidden state
        decoder_input = Variable(torch.LongTensor([0])) # EOS/SOS token
        if USE_CUDA:
            decoder_input = decoder_input.cuda()

        # Variables to store decoder and attention outputs
        decoder_outputs = Variable(torch.zeros(target_len, output_size))
        decoder_attentions = Variable(torch.zeros(target_len, input_len))
        if USE_CUDA:
            decoder_outputs = decoder_outputs.cuda()
            decoder_attentions = decoder_attentions.cuda()
        
        # Run through decoder
        for i in range(target_len):
            decoder_output, decoder_hidden, decoder_attention = self.decoder(context_embedded, decoder_input, decoder_hidden, encoder_outputs)
            decoder_outputs[i] = decoder_output
            decoder_attentions[i] = decoder_attention

            # Teacher forcing with known targets, if provided
            if word_targets is not None:
                decoder_input = word_targets[i]

            # Sample with last outputs
            else:
                max_index = decoder_output.topk(1)[1].data[0][0]
                decoder_input = Variable(torch.LongTensor([max_index]))
                if USE_CUDA:
                    decoder_input = decoder_input.cuda()

                if max_index == 0: break # EOS
        
        # Slice outputs
        if word_targets is None:
            decoder_outputs = decoder_outputs[:i]
            decoder_attentions = decoder_attentions[:i]
        else:
            decoder_attentions = decoder_attentions[:-1] # Ignore attentions on EOS

        return decoder_outputs, decoder_attentions

## Training

The inputs to the network are the current phrase label, e.g. `%getLightState` and the string to parse, "the living room light is on". The outputs are the child node labels, `$light` and `$on_off` with a selection of words given by attention-like weights over the sequence, treated as boolean values given a threshold.

In [18]:
input_size = len(input_tokens)
output_size = len(output_tokens)
hidden_size = 100

learning_rate = 1e-4
weight_decay = 1e-6

rarnn = RARNN(input_size, output_size, hidden_size)
optimizer = torch.optim.Adam(rarnn.parameters(), lr=learning_rate, weight_decay=weight_decay)

decoder_criterion = nn.NLLLoss()
attention_criterion = nn.MSELoss(size_average=False)

In [19]:
def train(flat, node):
    context, inputs, targets = data_for_node(flat, node)

    # Turn inputs into tensors
    context_var = tokens_to_tensor([context], output_tokens, False)
    context_var = Variable(context_var)
    inputs_var = tokens_to_tensor(inputs, input_tokens).view(-1, 1, 1) # seq x batch x size
    inputs_var = Variable(inputs_var)
    target_tokens = [target_token for target_token, _ in targets]
    target_ranges = [target_range for _, target_range in targets]
    target_tokens_var = tokens_to_tensor(target_tokens, output_tokens)
    target_tokens_var = Variable(target_tokens_var)
    target_ranges_var = ranges_to_tensor(target_ranges, len(inputs) + 1)
    target_ranges_var = Variable(target_ranges_var)
 
    # Run through model
    decoder_outputs, attention_outputs = rarnn(context_var, inputs_var, target_tokens_var)

    # Loss calculation and backprop
    optimizer.zero_grad()
    decoder_loss = decoder_criterion(decoder_outputs, target_tokens_var)
    attention_loss = attention_criterion(attention_outputs, target_ranges_var)
    total_loss = decoder_loss + attention_loss
    total_loss.backward()
    optimizer.step()

    return total_loss.data[0]

In [20]:
import sconce
job = sconce.Job('rarnn')
job.plot_every = 20
job.log_every = 100

n_epochs = 5000

for i in range(n_epochs):
    walked_flat, walked_tree = walk_tree(parsed, parsed['%'], None)
    def _train(node): return train(walked_flat, node)
    ds = descend(walked_tree, _train)
    d = sum(ds) / len(ds)
    job.record(i, d)

Starting job 5941d868f8e1c2083c22ccbb at 2017-06-14 17:44:24
[log] 0m 25s (100) 3.8491
[log] 0m 55s (200) 2.9594
[log] 1m 25s (300) 1.5643
[log] 1m 50s (400) 1.1570
[log] 2m 15s (500) 1.1597
[log] 2m 38s (600) 0.5652
[log] 3m 6s (700) 0.7624
[log] 3m 31s (800) 0.7045
[log] 3m 58s (900) 0.4503
[log] 4m 24s (1000) 1.0803
[log] 4m 51s (1100) 0.3412
[log] 5m 16s (1200) 0.0973
[log] 5m 41s (1300) 0.2184
[log] 6m 6s (1400) 0.1685
[log] 6m 30s (1500) 0.9838
[log] 6m 57s (1600) 0.2556
[log] 7m 24s (1700) 0.3182
[log] 7m 50s (1800) 0.3110
[log] 8m 15s (1900) 0.0274
[log] 8m 41s (2000) 0.0948
[log] 9m 6s (2100) 0.0605
[log] 9m 32s (2200) 0.0580
[log] 9m 56s (2300) 0.0305
[log] 10m 21s (2400) 0.0478
[log] 10m 45s (2500) 0.0406
[log] 11m 10s (2600) 0.0576
[log] 11m 35s (2700) 0.1102
[log] 11m 57s (2800) 0.0742
[log] 12m 23s (2900) 0.0177
[log] 12m 48s (3000) 0.0376
[log] 13m 13s (3100) 0.1005
[log] 13m 36s (3200) 0.0164
[log] 14m 2s (3300) 0.3248
[log] 14m 27s (3400) 0.0041
[log] 14m 53s (3500) 0.

KeyboardInterrupt: 

## Evaluating

In [82]:
def evaluate(context, inputs, node=None):
    if node == None:
        node = Node('parsed')
        node.position = (0, len(inputs) - 1)
    
    # Turn data into tensors
    context_var = tokens_to_tensor([context], output_tokens, False)
    context_var = Variable(context_var)
    inputs_var = tokens_to_tensor(inputs, input_tokens).view(-1, 1, 1) # seq x batch x size
    inputs_var = Variable(inputs_var)
    
    # Run through RARNN
    decoder_outputs, attention_outputs = rarnn(context_var, inputs_var)
    
    # Given the decoder and attention outputs, gather contexts and inputs for sub-phrases
    # Use attention values > 0.5 to select words for next input sequence

    next_contexts = []
    next_inputs = []
    next_positions = []
    
    for i in range(len(decoder_outputs)):
        max_value, max_index = decoder_outputs[i].topk(1)
        max_index = max_index.data[0]
        next_contexts.append(output_tokens[max_index]) # Get decoder output token
        a = attention_outputs[i]
        next_input = []
        next_position = []
        for t in range(len(a) - 1):
            at = a[t].data[0]
            if at > 0.5:
                if len(next_position) == 0: # Start position
                    next_position.append(t)
                next_input.append(inputs[t])
            else:
                if len(next_position) == 1: # End position
                    next_position.append(t - 1)
        if len(next_position) == 1: # End position
            next_position.append(t)
        next_inputs.append(next_input)
        next_position = (next_position[0] + node.position[0], next_position[1] + node.position[0])
        next_positions.append(next_position)

    evaluated = list(zip(next_contexts, next_inputs, next_positions))

    # Print decoded outputs
    print('\n(evaluate) %s %s -> %s' % (context, ' '.join(inputs), next_contexts))
    
#     # Plot attention outputs
#     fig = plt.figure(figsize=(len(inputs) / 3, 99))
#     sub = fig.add_subplot(111)
#     sub.matshow(attention_outputs.data.squeeze(1).numpy(), vmin=0, vmax=1, cmap='hot')
#     plt.show(); plt.close()
    
    for context, inputs, position in evaluated:
        # Add a node for parsed sub-phrases and values
        sub_node = Node(context)
        sub_node.position = position
        node.add(sub_node)
        
        # Recursively evaluate sub-phrases
        if context[0] == '%':
            if len(inputs) > 0:
                evaluate(context, inputs, sub_node)
            else:
                print("WARNING: Empty inputs")
    
        # Or add words directly to value node
        elif context[0] == '$':
            sub_node.add(' '.join(inputs))

    return node

In [83]:
def evaluate_and_print(context, inputs):
    evaluated = evaluate(context, inputs)
    print(' '.join(inputs))
    print(evaluated)
    return evaluated

In [84]:
walked_flat, walked_tree = walk_tree(parsed, parsed['%'], None)
inputs = [child.key for child in walked_flat.children]
print(inputs, walked_tree)
evaluate_and_print('%', inputs)

['uh', 'plx', 'btc', 'price', 'thanks', 'maia'] ( %
    ( %sequence (1, 3, 3)
        ( %action (2, 3, 2)
            ( %getPrice (2, 3, 2)
                ( $asset (2, 2, 1) btc ) ) ) ) )

(evaluate) % uh plx btc price thanks maia -> ['%sequence']

(evaluate) %sequence plx btc price -> ['%action']

(evaluate) %action btc price -> ['%getPrice']

(evaluate) %getPrice btc price -> ['$asset']
uh plx btc price thanks maia
( parsed (0, 5)
    ( %sequence (1, 3)
        ( %action (2, 3)
            ( %getPrice (2, 3)
                ( $asset (2, 2) btc ) ) ) ) )


In [85]:
evaluate_and_print('%', "hey maia if the ethereum price is less than 2 0 then turn the living room light on".split(' '))


(evaluate) % hey maia if the ethereum price is less than 2 0 then turn the living room light on -> ['%if']

(evaluate) %if if the ethereum price is less than 2 0 then turn the living room light on -> ['%condition', '%sequence']

(evaluate) %condition the ethereum price is less than 2 0 -> ['%getValue', '$operator', '$number']

(evaluate) %getValue the ethereum price -> ['%getPrice']

(evaluate) %getPrice the ethereum price -> ['$asset']

(evaluate) %sequence turn the living room light on -> ['%action']

(evaluate) %action turn the living room light on -> ['%setLightState']

(evaluate) %setLightState turn the living room light on -> ['$light_name', '$on_off']
hey maia if the ethereum price is less than 2 0 then turn the living room light on
( parsed (0, 17)
    ( %if (2, 17)
        ( %condition (3, 10)
            ( %getValue (3, 5)
                ( %getPrice (3, 5)
                    ( $asset (4, 4) ethereum ) ) )
            ( $operator (7, 8) less than )
            ( $number (9,

In [86]:
evaluate_and_print('%', "hey maia what's the ethereum price".split(' '))


(evaluate) % hey maia what's the ethereum price -> ['%sequence']

(evaluate) %sequence what's the ethereum price -> ['%action']

(evaluate) %action what's the ethereum price -> ['%getPrice']

(evaluate) %getPrice what's the ethereum price -> ['$asset']
hey maia what's the ethereum price
( parsed (0, 5)
    ( %sequence (2, 5)
        ( %action (2, 5)
            ( %getPrice (2, 5)
                ( $asset (4, 4) ethereum ) ) ) ) )


In [87]:
evaluate_and_print('%', "hey maia play some Skrillex please and then turn the office light off".split(' '))


(evaluate) % hey maia play some Skrillex please and then turn the office light off -> ['%sequence']

(evaluate) %sequence play some Skrillex please and then turn the office light off -> ['%action', '%action']

(evaluate) %action play some Skrillex -> ['%playMusic']

(evaluate) %playMusic play some Skrillex -> ['$artist_name']

(evaluate) %action turn the office light off -> ['%setLightState']

(evaluate) %setLightState turn the office light off -> ['$light_name', '$on_off']
hey maia play some Skrillex please and then turn the office light off
( parsed (0, 12)
    ( %sequence (2, 12)
        ( %action (2, 4)
            ( %playMusic (2, 4)
                ( $artist_name (4, 4) Skrillex ) ) )
        ( %action (8, 12)
            ( %setLightState (8, 12)
                ( $light_name (10, 11) office light )
                ( $on_off (12, 12) off ) ) ) ) )


In [88]:
evaluate_and_print('%', "turn the office light up and also could you please turn off the living room light and make the temperature of the bedroom to 6 thank you maia".split(' '))


(evaluate) % turn the office light up and also could you please turn off the living room light and make the temperature of the bedroom to 6 thank you maia -> ['%sequence']

(evaluate) %sequence turn the office light up and also could you please turn off the living room light and make the temperature of the bedroom to 6 -> ['%action', '%action', '%action']

(evaluate) %action turn the office light up -> ['%setLightState']

(evaluate) %setLightState turn the office light up -> ['$light_name', '$up_down']

(evaluate) %action turn off the living room light -> ['%setLightState']

(evaluate) %setLightState turn off the living room light -> ['$on_off', '$light_name']

(evaluate) %action make the temperature of the bedroom to 6 -> ['%setTemperature']

(evaluate) %setTemperature make the temperature of the bedroom to 6 -> ['$room_name', '$temperature']
turn the office light up and also could you please turn off the living room light and make the temperature of the bedroom to 6 thank you maia
(

In [89]:
evaluate_and_print('%', "turn the living room light off and turn the bedroom light up and also turn the volume up".split(' '))


(evaluate) % turn the living room light off and turn the bedroom light up and also turn the volume up -> ['%sequence']

(evaluate) %sequence turn the living room light off and turn the bedroom light up and also turn the volume up -> ['%action', '%action', '%action']

(evaluate) %action turn the living room light off -> ['%setLightState']

(evaluate) %setLightState turn the living room light off -> ['$light_name', '$on_off']

(evaluate) %action turn the bedroom light up -> ['%setLightState']

(evaluate) %setLightState turn the bedroom light up -> ['$light_name', '$up_down']

(evaluate) %action turn the volume up -> ['%setVolume']

(evaluate) %setVolume turn the volume up -> ['$up_down']
turn the living room light off and turn the bedroom light up and also turn the volume up
( parsed (0, 17)
    ( %sequence (0, 17)
        ( %action (0, 5)
            ( %setLightState (0, 5)
                ( $light_name (2, 4) living room light )
                ( $on_off (5, 5) off ) ) )
        ( %ac

In [123]:
def prepare_string(s):
    s = re.sub(r'(\d)', r'\1 ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.split(' ')

def parse(s, cb):
    words = prepare_string(s)
    try:
        evaluated = evaluate_and_print('%', words)
        cb({'words': words, 'parsed': evaluated.to_json()})
    except Exception:
        cb({'error': "Failed to evaluate"})

parse('hey maia if the price of bitcoin is greater than 3 0 0 0 turn the office light green', lambda r: print("response", r))


(evaluate) % hey maia if the price of bitcoin is greater than 3 0 0 0 turn the office light green -> ['%if']

(evaluate) %if if the price of bitcoin is greater than 3 0 0 0 turn the office light green -> ['%condition', '%sequence']

(evaluate) %condition the price of bitcoin is greater than 3 0 0 -> ['%getValue', '$operator', '$number']

(evaluate) %getValue the price of bitcoin -> ['%getPrice']

(evaluate) %getPrice the price of bitcoin -> ['$asset']

(evaluate) %sequence turn the office light green -> ['%action']

(evaluate) %action turn the office light green -> ['%setLightState']

(evaluate) %setLightState turn the office light green -> ['$light_name', '$color']
hey maia if the price of bitcoin is greater than 3 0 0 0 turn the office light green
( parsed (0, 18)
    ( %if (2, 18)
        ( %condition (3, 12)
            ( %getValue (3, 6)
                ( %getPrice (3, 6)
                    ( $asset (6, 6) bitcoin ) ) )
            ( $operator (8, 9) greater than )
            (

In [124]:
import somata
service = somata.Service('maia:parser', {'parse': parse}, {'bind_port': 8855})

Registered {'name': 'maia:parser', 'id': 'maia:parser~efhltonk', 'heartbeat': 0, 'port': 8855, 'client_id': 'efrxkpek'}

(evaluate) % if the price of eth is 9 9 then please turn on the kitchen light -> ['%if']

(evaluate) %if if the price of eth is 9 9 then please turn on the kitchen light -> ['%condition', '%sequence']

(evaluate) % if the price of eth is equal to 9 9 then please turn on the kitchen light -> ['%if']

(evaluate) %if if the price of eth is equal to 9 9 then please turn on the kitchen light -> ['%condition', '%sequence']

(evaluate) %condition the price of eth is equal to 9 9 -> ['%getValue', '$operator', '$number']

(evaluate) %getValue the price of eth -> ['%getPrice']

(evaluate) %getPrice the price of eth -> ['$asset']

(evaluate) %sequence please turn on the kitchen light -> ['%action']

(evaluate) %action turn on the kitchen light -> ['%setLightState']

(evaluate) %setLightState turn on the kitchen light -> ['$on_off', '$light_name']
if the price of eth is equal to

In [125]:
service.socket.close()